## 1. Importing libraries

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


---

## Reading The Data

In [2]:
project_dir="/home/spynom/project/flights_sageaker_project"
data_dir="data"

In [3]:
def get_data(project_dir:str,data_dir:str,filename:str)->pd.DataFrame:
    file_name=f"{filename}.csv"
    file_path= os.path.join(project_dir,data_dir,file_name)
    return pd.read_csv(file_path)

In [4]:
df=get_data(project_dir=project_dir,data_dir=data_dir,filename="raw")
df.sample(5)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
8027,Jet Airways,18/05/2019,Kolkata,Banglore,CCU → BOM → BLR,14:05,19:50,5h 45m,1 stop,No info,14781
841,Jet Airways,15/06/2019,Delhi,Cochin,DEL → BOM → COK,14:00,19:00,5h,1 stop,In-flight meal not included,10262
4300,Air India,21/05/2019,Delhi,Cochin,DEL → GOI → BOM → COK,22:00,19:15 22 May,21h 15m,2 stops,No info,10231
2958,SpiceJet,21/04/2019,Banglore,Delhi,BLR → DEL,05:55,08:35,2h 40m,non-stop,No info,4423
2467,Jet Airways,3/06/2019,Mumbai,Hyderabad,BOM → HYD,19:35,21:05,1h 30m,non-stop,No info,5678


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- Dataset has total 10683 instances and 10 features.
- Two features have one missing value in each.
- Data type of some features is inappropriate.

---

## Preliminary Analysis

### Check Data Types

In [6]:
df.sample(5)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
6700,SpiceJet,1/05/2019,Kolkata,Banglore,CCU → BLR,09:00,11:25,2h 25m,non-stop,No info,4174
10122,Air India,6/06/2019,Delhi,Cochin,DEL → MAA → COK,06:05,09:25 07 Jun,27h 20m,1 stop,No info,7952
756,Air India,18/05/2019,Delhi,Cochin,DEL → BOM → COK,03:50,19:15,15h 25m,1 stop,No info,8879
2445,Multiple carriers,18/05/2019,Delhi,Cochin,DEL → BOM → COK,06:00,19:15,13h 15m,1 stop,No info,8476


In [7]:
df.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

### Check Duplicates

In [8]:
df.duplicated().sum()

np.int64(220)

In [9]:
df.loc[df.duplicated(keep=False)].sort_values(["Date_of_Journey"])

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
7502,SpiceJet,03/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No info,6860
6196,IndiGo,03/03/2019,Banglore,New Delhi,BLR → DEL,08:30,11:30,3h,non-stop,No info,6860
10275,IndiGo,03/03/2019,Banglore,New Delhi,BLR → DEL,08:30,11:30,3h,non-stop,No info,6860
...,...,...,...,...,...,...,...,...,...,...,...
1312,Jet Airways,9/06/2019,Delhi,Cochin,DEL → NAG → BOM → COK,06:45,12:35 10 Jun,29h 50m,2 stops,No info,13376
9191,Jet Airways,9/06/2019,Delhi,Cochin,DEL → JAI → BOM → COK,09:40,12:35 10 Jun,26h 55m,2 stops,In-flight meal not included,10368
9216,Jet Airways,9/06/2019,Delhi,Cochin,DEL → IDR → BOM → COK,05:30,04:25 10 Jun,22h 55m,2 stops,No info,13292
8446,Jet Airways,9/06/2019,Delhi,Cochin,DEL → NAG → BOM → COK,06:45,12:35 10 Jun,29h 50m,2 stops,No info,13376


### observations:

- The data type of `Date_of_Journey`, `Dep_Time` and `Arrival_Time` need to be changed to datetime.
- The  type of `Duration` and `Total_Stops` is mixed. It should be numeric type.
- There are 220 duplicates, need to be removed.

---

## Detail Analysis

In [15]:
df.columns

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info', 'Price'],
      dtype='object')

**Airline**

In [23]:
df.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

**Date_of_Journey**

In [22]:
df.Date_of_Journey.dtype

dtype('O')

**Source**

In [28]:
df.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

**Destination**

In [29]:
df.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

**Dep_Time**

In [31]:
df.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [35]:
(
    df
    .Dep_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
)

Series([], Name: Dep_Time, dtype: object)

In [37]:
pd.to_datetime(df.Dep_Time).dt.time

/tmp/ipykernel_37467/3599857698.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df.Dep_Time).dt.time


0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

**Arrival_Time**

In [44]:
(
    df
    .Arrival_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
    .str.split(' ',n=1)
    .str.get(-1).unique()
)

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

**Duration**

In [56]:
(df.Duration
.loc[lambda ser: ~ser.str.contains("h")]
#.astype(np.int32)*60

)

6474    5m
Name: Duration, dtype: object

In [57]:
df.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


- The observation indexed 6474 has duration of 5 minutes. This is clearly worng. Will delete this observation.

In [65]:
(
    df
    .Duration
    .drop(index=6474)
    .str.split(" ", expand=True)
    .set_axis(["hour","minute"],axis=1)
    .assign(
        hour=lambda df_: (
            df_
            .hour
            .str.replace("h","")
            .astype(int)
            .mul(60)),
        
        minute = lambda df_: (
            df_
            .minute
            .str.replace("m","")
            .fillna(0)
            .astype(int)
            
        

        )
    ).sum(axis=1)
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

**Total_Stops**

In [80]:
df.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [85]:
(
    df.Total_Stops
    .replace("non-stop","0")
    .str.replace("stops?","",regex=True)#"stop" or "stops"
    .str.strip()
    .pipe(lambda ser: pd.to_numeric(ser))
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

**Additional_Info**

In [96]:
(df
 .Additional_Info
 .replace("No info","No Info")
 .unique()
)

array(['No Info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

---

## Cleaning Operations

In [108]:
def clean_data(df:pd.DataFrame)->pd.DataFrame:
    return(
        df
        .drop(index=6474)
        .assign(**{
            col:df[col].str.strip()
            for col in df.select_dtypes(include="O").columns
        })
        .assign(
        Airline=df.Airline
        .str.replace(" Premium economy","")
        .str.replace(" Business",""),
        )
        
        .rename(columns=str.lower)
        .assign(
            date_of_journey=lambda df_: (
            pd.to_datetime( df_.date_of_journey,dayfirst=True)
        ),
        dep_time= lambda df_: pd.to_datetime(df_.dep_time).dt.time,
        arrival_time= lambda df_: pd.to_datetime(df_.arrival_time).dt.time,
        duration= lambda df_: df_.duration.pipe(convert_to_minutes),
        total_stops= lambda df_: (
                df_.total_stops
                .replace("non-stop","0")
                .str.replace("stops?","",regex=True)#"stop" or "stops"
                .str.strip()
                .pipe(lambda ser: pd.to_numeric(ser))),
        additional_info= lambda df_: df_
                                .additional_info
                                .replace("No info","No Info")
        )
        .drop(columns="route")
        .drop_duplicates()
        
    )

In [74]:
def convert_to_minutes(duration):
    return (
    duration
    .str.split(" ", expand=True)
    .set_axis(["hour","minute"],axis=1)
    .assign(
        hour=lambda df_: (
            df_
            .hour
            .str.replace("h","")
            .astype(int)
            .mul(60)),
        
        minute = lambda df_: (
            df_
            .minute
            .str.replace("m","")
            .fillna(0)
            .astype(int)
            
        

        )
    ).sum(axis=1)
)

In [110]:
final_data=clean_data(df)

/tmp/ipykernel_37467/3092866008.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time= lambda df_: pd.to_datetime(df_.dep_time).dt.time,
/tmp/ipykernel_37467/3092866008.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time= lambda df_: pd.to_datetime(df_.arrival_time).dt.time,


----

## Data split

In [118]:
def split(X,y,test_size=0.1,val_size=0.2):
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=(1-test_size-val_size), random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=(round((test_size/(test_size+val_size)),2)), random_state=42)
    return X_train,X_val,X_test,y_train,y_val,y_test

In [119]:
X=final_data.drop(columns=["price"])

In [120]:
y=final_data.price

In [121]:
X_train,X_val,X_test,y_train,y_val,y_test= split(X,y)

In [122]:
X_train.shape

(7321, 9)

In [123]:
X_val.shape

(2103, 9)

In [124]:
X_test.shape

(1036, 9)

In [127]:
def export_data(X, y, name):
	file_name = f"{name}.csv"
	file_path = os.path.join(project_dir, data_dir, file_name)

	X.join(y).to_csv(file_path, index=False)

	return pd.read_csv(file_path).head()

In [128]:
export_data(X_train, y_train, "train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,GoAir,2019-05-24,Banglore,Delhi,11:40:00,14:30:00,170,0.0,No Info,3398
1,SpiceJet,2019-05-18,Chennai,Kolkata,09:45:00,12:00:00,135,0.0,No check-in baggage included,3543
2,Air Asia,2019-04-03,Kolkata,Banglore,10:20:00,12:55:00,155,0.0,No Info,4107
3,Jet Airways,2019-04-06,Delhi,Cochin,08:00:00,19:00:00,660,1.0,In-flight meal not included,5406
4,IndiGo,2019-05-27,Banglore,Delhi,16:55:00,19:55:00,180,0.0,No Info,4823


In [129]:
export_data(X_val, y_val, "val")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,SpiceJet,2019-06-06,Mumbai,Hyderabad,22:45:00,00:15:00,90,0.0,No check-in baggage included,1965
1,Jet Airways,2019-03-21,Banglore,New Delhi,08:55:00,15:15:00,380,1.0,In-flight meal not included,7832
2,Jet Airways,2019-06-03,Delhi,Cochin,19:45:00,19:00:00,1395,1.0,No Info,14714
3,Jet Airways,2019-05-21,Kolkata,Banglore,18:55:00,04:40:00,585,1.0,No Info,14388
4,Air India,2019-05-15,Kolkata,Banglore,09:25:00,18:20:00,535,2.0,No Info,8366


In [130]:
export_data(X_test, y_test, "test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-05-12,Banglore,Delhi,19:50:00,22:50:00,180,0.0,No Info,7229
1,Air India,2019-04-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,5562
2,Multiple carriers,2019-06-01,Delhi,Cochin,12:50:00,21:00:00,490,1.0,No Info,7005
3,Multiple carriers,2019-03-06,Delhi,Cochin,17:00:00,01:35:00,515,1.0,No Info,24318
4,Jet Airways,2019-05-06,Kolkata,Banglore,21:10:00,08:15:00,665,1.0,In-flight meal not included,7757


------